In [1]:
import sys
sys.path.append('..')

import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
from datetime import datetime
from sqlalchemy import create_engine, text

from utilities_amigocloud import AmigocloudFunctions

In [2]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
ID_PROYECTO = 35248

In [3]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [4]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_amigocloud():
    query = '''SELECT parte.*, plan.unidad_01 
                FROM dataset_367119 parte 
                JOIN dataset_360912 plan 
                ON ST_Intersects(parte.referencia, plan.geometry);'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def obtener_propiedad_plan_amigocloud(cod_prop):
    query = f'''SELECT * FROM dataset_360912 WHERE unidad_01 = {cod_prop}'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def actualizar_lotes_db(unidad_01, unidad_03, ors, row):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    idd= :id,
                    fecha = :fecha,
                    semana = :semana,
                    temp_max = :temp_max,
                    temp_min = :temp_min,
                    temp = :temp,
                    viento = :viento,
                    humedad = :humedad,
                    cultivo = :tipo_cultivo,
                    desarrollo = :tipo_desarrollo
                WHERE unidad_01 = :unidad_01
                  AND unidad_03 = :unidad_03
                  AND os = :ors
            """)
            conn.execute(query, {
                "unidad_01": unidad_01,
                "unidad_03": unidad_03,
                "ors": ors,
                "id": row.id,
                "fecha": row.fecha_registro,
                "semana": datetime.fromisoformat(row.fecha_registro).isocalendar().week,
                "temp_max": row.temp_max,
                "temp_min": row.temp_min,
                "temp": (row.temp_max + row.temp_min) / 2,
                "viento": row.viento,
                "humedad": row.humedad,
                "tipo_cultivo": row.tipo_cultivo,
                "tipo_desarrollo": row.tipo_desarrollo
            })
            print(f"✔️ Registros con unidad_01={unidad_01}, unidad_03={unidad_03}, os={ors} marcados como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def copiar_parte_diario_a_clon_amigocloud(id):
    query = f"""INSERT INTO dataset_367584 (id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia) 
                SELECT id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia 
                FROM dataset_367119 
                WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_parte_diario_amigocloud(id):
    query = f"""DELETE FROM dataset_367119 WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def copiar_planificacion_a_clon_amigocloud(unidad_01, ors):
    query = f"""INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry) 
                SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry 
                FROM dataset_360912 
                WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_planificacion_amigocloud(unidad_01, ors):
    query = f"""DELETE FROM dataset_360912 WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def convertir_json_a_geodataframe(json_data, nom_campo_geo):
    # Convertimos a DataFrame para manipulación tabular
    df = pd.DataFrame(json_data)
    # Convertimos la columna geometry (WKB hexadecimal) a objetos shapely
    df[nom_campo_geo] = df[nom_campo_geo].apply(lambda x: wkb.loads(bytes.fromhex(x)))
    # Creamos el GeoDataFrame en CRS original (EPSG:4326)
    gdf = gpd.GeoDataFrame(df, geometry=nom_campo_geo, crs='EPSG:4326')
    # Reproyectamos a UTM Zona 20S (EPSG:32720)
    gdf_utm20s = gdf.to_crs(epsg=32720)
    return gdf_utm20s

In [5]:
parte_diario = obtener_parte_diario_amigocloud()

In [6]:
parte_diario

[{'id': 68,
  'amigo_id': '5fb4dfe403d44c70a3f3214c320c29e6',
  'fecha_registro': '2025-08-21 18:51:56+00:00',
  'referencia': '0101000020E61000003AED84F299A44FC056647440126631C0',
  'temp_max': 31.0,
  'temp_min': 29.0,
  'viento': 4.0,
  'humedad': 47.0,
  'observaciones': None,
  'tipo_cultivo': 'CAÑA DE AZUCAR',
  'tipo_desarrollo': 'CRECIMIENTO',
  'tipo': 'PARTE',
  'unidad_01': 50}]

In [7]:
df_parte_diario = convertir_json_a_geodataframe(parte_diario, 'referencia')

In [8]:
df_parte_diario

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo,tipo,unidad_01
0,68,5fb4dfe403d44c70a3f3214c320c29e6,2025-08-21 18:51:56+00:00,POINT (469627.748 8076313.480),31.0,29.0,4.0,47.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,50


In [9]:
for i, row in df_parte_diario.iterrows():
    cod_prop = row.unidad_01
    prop = obtener_propiedad_plan_amigocloud(cod_prop)
    gdf_prop = convertir_json_a_geodataframe(prop, 'geometry')
    # se queda con las combinacion unicas de unidad_01, unidad_03 y os
    gdf_prop_unicas = gdf_prop.drop_duplicates(subset=["unidad_01", "unidad_03", "os"])
    for j, row2 in gdf_prop_unicas.iterrows():
        print(row2.os)
        actualizar_lotes_db(row2.unidad_01, row2.unidad_03, row2.os, row)
        copiar_planificacion_a_clon_amigocloud(row2.unidad_01, row2.os)
        eliminar_planificacion_amigocloud(row2.unidad_01, row2.os)
    copiar_parte_diario_a_clon_amigocloud(row.id)
    eliminar_parte_diario_amigocloud(row.id)
    print(f'SE PROCESO PROPIEDAD: {row2.unidad_02}')

60
✔️ Registros con unidad_01=50, unidad_03=5373, os=60 marcados como procesado.
61
✔️ Registros con unidad_01=50, unidad_03=5373, os=61 marcados como procesado.
SE PROCESO PROPIEDAD: EL CIDRAL
